In [91]:
from __future__ import print_function

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.datasets import imdb

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Embedding
max_features = 1000000
maxlen = 10  #等同Timesteps
embedding_size = 128

# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 32
epochs = 100

In [92]:
text_file = open("./SelfData/U08_ONU16_15S_Ai_635745_0516.txt","r")#ONU_16_GrantDownLoadSizePerCycle, ONU[27]_GrantDown.txt
lines = text_file.readlines()
i = 1

Down64GB = []

i = 0
for line in lines:
    if i < 5000:
        n = float(line)
        if(n > 195000.0):
            Down64GB.append(195000.0)
        elif n >= 0:
            Down64GB.append(float(line))
    i = i + 1

print(len(Down64GB))
print(Down64GB[20:40])
print('max: ',max(Down64GB))

5000
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12390.0, 11959.0, 10527.0, 41509.0, 1107.0, 36682.0, 27262.0, 3326.0, 4707.0]
max:  195000.0


In [93]:
#data_array = Down64GB[0:2000]
#data_array_2D = np.array(data_array).reshape(200, 10)
#data_array_2D[0:5]

In [94]:
data = []
data_size = len(Down64GB)     ####
train_size = data_size - 1000  ### 
test_size = 1000   ###8G-1 = 7456, 8G-2 = 7450 
timesteps_ = 10               #
print('data_size : ',data_size)
print('train_size : ',train_size)
print('test_size : ',test_size)

for info in Down64GB:         ####
    data.append(info) 

data = np.array(data) 

data_size :  5000
train_size :  4000
test_size :  1000


In [95]:
TrainSet, TestSet = np.zeros(train_size), np.zeros(test_size)
TrainSet, TestSet = data[0:len(TrainSet)], data[len(TrainSet):data.shape[0]]
TrainSet, TestSet = np.array(TrainSet), np.array(TestSet)
TrainSet, TestSet = np.reshape(TrainSet,(len(TrainSet),1)), np.reshape(TestSet,(len(TestSet),1))

In [96]:
X_train = []
Y_train = []

period = maxlen
end_of_TrainSet = train_size

print('type(TrainSet) : ',type(TrainSet))
print('TrainSet.shape : ',TrainSet.shape)

#X_train = TrainSet[:]
#Y_train = TrainSet[period:]
for i in range(1,end_of_TrainSet): #
    X_train.append(TrainSet[i-1:i,0])
    Y_train.append(TrainSet[i,0])

print('type(X_train) : ',type(X_train))
print('type(Y_train) : ',type(Y_train))    
    
X_train, Y_train = np.array(X_train), np.array(Y_train)
#X_train
#X_train.shape

type(TrainSet) :  <class 'numpy.ndarray'>
TrainSet.shape :  (4000, 1)
type(X_train) :  <class 'list'>
type(Y_train) :  <class 'list'>


In [97]:
X_test = []
Y_test = []

real_value = []
end_of_TestSet = test_size

print('type(TestSet) : ',type(TestSet))
print('TestSet.shape : ',TestSet.shape)

#X_test = TestSet[:]
#Y_test = TestSet[period:]
for i in range(1,end_of_TestSet):
    X_test.append(TestSet[i-1:i,0])
    Y_test.append(TestSet[i,0])
    
print('type(X_test) : ',type(X_test))
print('type(Y_test) : ',type(Y_test))

X_test, Y_test = np.array(X_test), np.array(Y_test)
#print('type(X_test) : ',type(X_test))
#X_test
#X_test.shape

type(TestSet) :  <class 'numpy.ndarray'>
TestSet.shape :  (1000, 1)
type(X_test) :  <class 'list'>
type(Y_test) :  <class 'list'>


In [98]:
print('Loading data...')
#(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = X_train
x_test = X_test
y_train = Y_train
y_test = Y_test

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
#y_train = sequence.pad_sequences(y_train, maxlen=maxlen)
#y_test = sequence.pad_sequences(y_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Loading data...
3999 train sequences
999 test sequences
Pad sequences (samples x time)
x_train shape: (3999, 10)
x_test shape: (999, 10)
y_train shape: (3999,)
y_test shape: (999,)


In [99]:
print('Build model...')

model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(Dropout(0.25))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Build model...
Train...
Epoch 1/100


UnknownError: 2 root error(s) found.
  (0) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_6/conv1d_6/conv1d (defined at <ipython-input-99-3c30d2d36aef>:24) ]]
  (1) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_6/conv1d_6/conv1d (defined at <ipython-input-99-3c30d2d36aef>:24) ]]
	 [[gradient_tape/sequential_6/embedding_6/embedding_lookup/Reshape/_38]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_18223]

Function call stack:
train_function -> train_function


In [ ]:
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)